# Low Rank Matrix Factorization

## Formulation
Let's assume that our system has $I_{user}$ users and $J_{movie}$ movies. We assign $K_{latent}$ features to each user and movie in the system. We can construct a matrix factorization as follows:

$$
\begin{vmatrix}
x_{0,0} & x_{0,1} & x_{0, 2} & ... & x_{0, K} \\
x_{1,0} & ...     & ...      & ... & ...      \\
x_{2,0} & ...     & ...      & ... & ...      \\
...     & ...     & ...      & ... & ...      \\
x_{I,0} & ...     & ...      & ... & x_{I, K}
\end{vmatrix}
\begin{vmatrix}
\theta_{0,0} & \theta_{0,1} & \theta_{0, 2} & ... & \theta_{0, K} \\
\theta_{1,0} & ...     & ...      & ... & ...      \\
\theta_{2,0} & ...     & ...      & ... & ...      \\
...     & ...     & ...      & ... & ...      \\
\theta_{J,0} & ...     & ...      & ... & \theta_{J, K}
\end{vmatrix}^{T}
=
\begin{vmatrix}
r_{0,0} & r_{0,1} & r_{0, 2} & ... & r_{0, J} \\
r_{1,0} & ...     & ...      & ... & ...      \\
r_{2,0} & ...     & ...      & ... & ...      \\
...     & ...     & ...      & ... & ...      \\
r_{I,0} & ...     & ...      & ... & r_{I, J}
\end{vmatrix}
$$

$$
X\Theta^{T} = \hat{R}
$$

$X$ represents the latent feature matrix for all users in our system. $\Theta$ represents the latent feature matrix for all all movies in our system. The matrix product of $X$ and $\Theta^{T}$ is the model predicated rating. Let $R$ represents the actual rating we received from the MovieLens dataset. For every missing value in $R$, we will place them with average rating each movie received from the poll of users. Then we define the loss function as follows:

$$
L_{X, \Theta} = \frac{1}{2}\Sigma_{i,j} (X\Theta^{T} - R)^{2} + \frac{\lambda}{2}\Sigma_{i, k}X^{2} + \frac{\lambda}{2}\Sigma_{j, k}\Theta^{2}
$$

The optimization objective here is to minimize the loss function above.

## Partial Derivatives & Gradients

Recall that the output of our low-rank matrices model is $\hat{R}$ and let's find the gradient of $L$ with respect to $\hat{R}$ first. The $\frac{1}{2}$ term will get canceled out by the square term.

$$
\frac{\partial L}{\partial \hat{R}} = \hat{R} - R
$$

$$
\hat{R} = X\Theta^{T}
$$

Now let's figure out the gradient of $\hat{R}$ with respect to $X$ and $\Theta$:

$$
\frac{\partial \hat{R}}{\partial X} = \Theta^{T}
$$

$$
\frac{\partial \hat{R}}{\partial \Theta} = X
$$

Using chain rule, we can then do the following:

$$
\frac{\partial L}{\partial X} = \frac{\partial L}{\partial \hat{R}}\frac{\partial \hat{R}}{\partial X}
$$

$$
\frac{\partial L}{\partial \Theta} = \frac{\partial L}{\partial \hat{R}}\frac{\partial \hat{R}}{\partial \Theta}
$$

In Python,
```python
# Denote U as the user latent feature matrix and M as the movie latent feature matrix
pred = np.dot(U, M.T)
grad_pred = pred - R 
grad_u = np.dot(grad_pred, M)+ (reg * U)
grad_m = np.dot(grad_pred.T, U) + (reg * M)
```

In [16]:
import numpy as np


def loss(U, M, R, reg=0.0):
    """Compute loss
    
    :param U: User latent feature matrix, there are I movies and K features
    :type U: numpy 2-d array
    :param M: Movie latent feature matrix, there are J movies and K features
    :type M: numpy 2-d array
    :param R: Rating matrix, i-index represents user and j-index represents movie
    :type R: numpy 2-d array
    :param reg: Regularization strength
    :type reg: float
    """
    diff = np.dot(U, M.T) - R
    loss = 0.5 * np.sum(diff * diff)
    loss += reg * np.sum(U * U) / 2
    loss += reg * np.sum(M * M) / 2
    return loss


def rel_error(X, Y):
    """Compute maximum relative error
    
    :param X: Matrix of the same shape as Y
    :type X: numpy array
    :param Y: Matrix of the same shape as X
    :type Y: numpy array
    """
    return np.max(np.abs(X - Y) / (np.maximum(1e-8, np.abs(X) + np.abs(Y))))


def compute_grad(U, M, R, reg=0.0):
    """Compute gradients for U and M
    
    :param U: User latent feature matrix, there are I movies and K features
    :type U: numpy 2-d array
    :param M: Movie latent feature matrix, there are J movies and K features
    :type M: numpy 2-d array
    :param R: Rating matrix, i-index represents user and j-index represents movie
    :type R: numpy 2-d array
    :param reg: Regularization strength
    :type reg: float
    """
    u_grad = np.zeros(U.shape)
    m_grad = np.zeros(M.shape)
    
    num_user, lat_dim = U.shape
    num_movie, lat_dim = M.shape
    
    diff = np.dot(U, M.T) - R
    for i in range(num_user):
        u_grad[i] = np.sum(diff[i].reshape(num_movie, 1) * M, axis=0) + (reg * U[i])

    for j in range(num_movie):
        m_grad[j] = np.sum(diff.T[j].reshape(num_user, 1) * U, axis=0) + (reg * M[j])
        
    return u_grad, m_grad


def compute_grad_vectorized(U, M, R, reg=0.0):
    """Compute gradients for U and M
    
    :param U: User latent feature matrix, there are I movies and K features
    :type U: numpy 2-d array
    :param M: Movie latent feature matrix, there are J movies and K features
    :type M: numpy 2-d array
    :param R: Rating matrix, i-index represents user and j-index represents movie
    :type R: numpy 2-d array
    :param reg: Regularization strength
    :type reg: float
    """
    grad_out = np.dot(U, M.T) - R 
    grad_u = np.dot(grad_out, M)+ (reg * U)
    grad_m = np.dot(grad_out.T, U) + (reg * M)
    return grad_u, grad_m


def compute_num_grad(U, M, R, loss_func, reg=0.0, h=1e-5):
    """Compute numerical gradients for U and M
    
    :param U: User latent feature matrix, there are I movies and K features
    :type U: numpy 2-d array
    :param M: Movie latent feature matrix, there are J movies and K features
    :type M: numpy 2-d array
    :param R: Rating matrix, i-index represents user and j-index represents movie
    :type R: numpy 2-d array
    :param reg: Regularization strength
    :type reg: float
    """
    num_grad_u = np.zeros(U.shape)
    num_grad_m = np.zeros(M.shape)
    
    U_dim, L_dim = U.shape
    M_dim, L_dim = M.shape
    
    for i in range(U_dim):
        for k in range(L_dim):
            old_val = U[i][k]
            
            U[i][k] = old_val + h
            fuph = loss_func(U, M, R, reg)
            
            U[i][k] = old_val - h
            fumh = loss_func(U, M, R, reg)
            
            U[i][k] = old_val
            num_grad_u[i][k] = (fuph - fumh) / (2 * h)
    
    for j in range(M_dim):
        for k in range(L_dim):
            old_val = M[j][k]
            
            M[j][k] = old_val + h
            fmph = loss_func(U, M, R, reg)
            
            M[j][k] = old_val - h
            fmmh = loss_func(U, M, R, reg)
            
            M[j][k] = old_val
            num_grad_m[j][k] = (fmph - fmmh) / (2 * h)
    
    return num_grad_u, num_grad_m

In [17]:
num_user = 3
num_movie = 3
lat_dim = 4
reg = 0.5

R = np.random.rand(num_user, num_movie)
U = np.random.rand(num_user, lat_dim)
M = np.random.randn(num_movie, lat_dim)

np.dot(U, M.T)

grad_u, grad_m = compute_grad(U, M, R, reg)
num_grad_u, num_grad_m = compute_num_grad(U, M, R, loss, reg)

print "Gradient of U relative error %s" % str(rel_error(grad_u, num_grad_u))
print "Gradient of M relative error %s" % str(rel_error(grad_m, num_grad_m))

print "\nFully vectorized implementation\n"
grad_u, grad_m = compute_grad_vectorized(U, M, R, reg)
print "Gradient of U relative error %s" % str(rel_error(grad_u, num_grad_u))
print "Gradient of M relative error %s" % str(rel_error(grad_m, num_grad_m))

Gradient of U relative error 2.5957030021162005e-10
Gradient of M relative error 8.20323831585007e-11

Fully vectorized implementation

Gradient of U relative error 2.595711175602241e-10
Gradient of M relative error 8.20323831585007e-11
